<a href="https://colab.research.google.com/github/hamzzihyun/coachingstudy-9_AI_basic/blob/main/Week5_mission_jihyun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Q1

import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader

training_epochs = 15 # training 반복 횟수
batch_size = 100

root = './data'
mnist_train = dset.MNIST (root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST (root=root, train=False, transform=transforms.ToTensor(), download=True)

# data loader를 직접 구현해보자.
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True) # 구현
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Q2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")

linear = torch.nn.Linear(784, 10, bias=True).to(device)

# weight init
torch.nn.init.normal_(linear.weight)

cpu is available


Parameter containing:
tensor([[-0.1953, -0.4404,  0.0139,  ...,  0.7510,  1.5190,  0.9637],
        [ 0.4146, -1.0660, -0.9969,  ..., -0.3259, -0.2695, -1.3316],
        [ 0.2600, -1.1821,  0.0065,  ...,  1.0648, -1.2251,  0.9841],
        ...,
        [ 0.0279, -1.7242, -1.1723,  ...,  1.3651, -0.0689, -0.7931],
        [ 0.0952,  0.1939,  0.1636,  ..., -1.0489,  0.6224, -1.7181],
        [ 2.6504,  0.6297, -1.6571,  ..., -0.0781, -1.1398, -0.8711]],
       requires_grad=True)

In [ ]:
# Q3

# Loss fn - Cross Entropy Loss
criterion = torch.nn.CrossEntropyLoss().to(device)

# optimizer - SGD
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [ ]:
# Q4

for epoch in range(training_epochs): # epochs = 15
    for i, (imgs, labels) in enumerate(train_loader):
      imgs, labels = imgs.to(device), labels.to(device)
      imgs = imgs.view(-1, 28*28)

      outputs = linear(imgs) 
      loss = criterion(outputs, labels) 

      optimizer.zero_grad() # 모든 model의 gradient 값을 0으로 설정
      loss.backward() # backworad 함수를 호출해 gradient 계산
      optimizer.step() # 모델의 학습 파라미터 갱신

      _,argmax = torch.max(outputs, 1)
      accuracy = (labels == argmax).float().mean()

      if (i+1) % 100 == 0:
          print('Epoch [{}/{}], Step [{}/{}], Loss: {: .4f}, Accuracy: {: .2f}%'.format(
              epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item()* 100))

Epoch [1/15], Step [100/600], Loss:  3.0828, Accuracy:  46.00%
Epoch [1/15], Step [200/600], Loss:  2.3795, Accuracy:  54.00%
Epoch [1/15], Step [300/600], Loss:  1.5419, Accuracy:  70.00%
Epoch [1/15], Step [400/600], Loss:  1.3058, Accuracy:  79.00%
Epoch [1/15], Step [500/600], Loss:  1.3530, Accuracy:  72.00%
Epoch [1/15], Step [600/600], Loss:  1.5805, Accuracy:  72.00%
Epoch [2/15], Step [100/600], Loss:  0.9366, Accuracy:  81.00%
Epoch [2/15], Step [200/600], Loss:  0.8934, Accuracy:  80.00%
Epoch [2/15], Step [300/600], Loss:  0.7445, Accuracy:  82.00%
Epoch [2/15], Step [400/600], Loss:  1.0087, Accuracy:  77.00%
Epoch [2/15], Step [500/600], Loss:  0.6499, Accuracy:  87.00%
Epoch [2/15], Step [600/600], Loss:  0.8846, Accuracy:  81.00%
Epoch [3/15], Step [100/600], Loss:  0.8999, Accuracy:  85.00%
Epoch [3/15], Step [200/600], Loss:  0.7794, Accuracy:  85.00%
Epoch [3/15], Step [300/600], Loss:  0.6670, Accuracy:  84.00%
Epoch [3/15], Step [400/600], Loss:  0.9762, Accuracy: 

In [ ]:
# Q5

# test
linear.eval() # evaluate mode로 전환 dropout 이나 batch_normalization 해제
with torch.no_grad(): # grad 해제
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs) # 구현

        _, argmax = torch.max(outputs, 1) # max()를 통해 최종 출력이 가장 높은 class 선택
        total += imgs.size(0) # 전체 클래스 개수
        correct += (labels == argmax). sum().item() # 예측값과 실제값이 같은지 비교

    print('Test accuracy for {} images: {: .2f}%'.format(total, correct / total * 100))

Test accuracy for 10000 images:  89.19%
